In [1]:
import pandas as pd
import numpy as np 
from igraph import Graph
import networkx as nx
from ast import literal_eval
import importlib
import steiner_tree

In [2]:
graph = Graph.Read_GML('../../data/processed/graph_apid_huri')
disgenet_protein_index = pd.read_csv('../../data/processed/disgenet_proteins_indexes_apid_huri.csv')
adj_matrix_apid_huri = np.load("../../data/processed/adjacency_matrix_apid_huri.npy")
ids = graph.vs['name']

In [12]:
disgenet_protein_index['protein_index'] = disgenet_protein_index['protein_index'].apply(literal_eval)
disgenet_protein_index['proteins_ids'] = disgenet_protein_index['proteins_ids'].apply(literal_eval)

In [13]:
importlib.reload(steiner_tree)
from tqdm.notebook import tqdm
tqdm.pandas()
disgenet_protein_index['main_component'], disgenet_protein_index['conservative_module'], disgenet_protein_index['added_nodes'] = disgenet_protein_index.progress_apply(lambda row: steiner_tree.sca(row['protein_index'],graph,adj_matrix_apid_huri), axis=1, result_type='expand').T.values

  0%|          | 0/301 [00:00<?, ?it/s]

In [15]:
disgenet_protein_index['len_main_component'] = disgenet_protein_index['main_component'].apply(lambda x: len(x))
disgenet_protein_index['len_cm'] = disgenet_protein_index['conservative_module'].apply(lambda x: len(x))
disgenet_protein_index['len_added_nodes'] = disgenet_protein_index['added_nodes'].apply(lambda x: len(x))

In [ ]:
disgenet_protein_index['main_component_ids'] = disgenet_protein_index.apply(lambda row: [ids[i] for i in row['main_component']], axis=1)
disgenet_protein_index['conservative_module_ids'] = disgenet_protein_index.apply(lambda row: [ids[i] for i in row['conservative_module']], axis=1)
disgenet_protein_index['added_nodes_ids'] = disgenet_protein_index.apply(lambda row: [ids[i] for i in row['added_nodes']], axis=1)

In [18]:
disgenet_protein_index.to_csv('../../data/processed/disgenet_prot_index_main_comp.csv', index=False)